In [1]:
import numpy as np
import pandas as pd
import os 
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import pickle
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
main_df = pd.read_csv('main_df.csv',dtype={2:'str',4:'str'})

In [3]:
main_df = main_df.drop(['Unnamed: 0'],axis = 1)

In [4]:
main_df

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,Benign,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,Benign,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,Benign,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,Benign,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,...,0,2,4,2,3,1,1,2,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540038,59.166.0.5,33094,149.171.126.7,43433,tcp,FIN,0.087306,320,1828,31,...,0,1,2,3,3,1,1,3,Benign,0
2540039,59.166.0.7,20848,149.171.126.4,21,tcp,CON,0.365058,456,346,31,...,2,2,2,2,2,2,2,2,Benign,0
2540040,59.166.0.3,21511,149.171.126.9,21,tcp,CON,6.335154,1802,2088,31,...,2,2,2,4,2,2,2,2,Benign,0
2540041,59.166.0.9,35433,149.171.126.0,80,tcp,CON,2.200934,3498,166054,31,...,0,1,1,2,4,2,2,2,Benign,0


In [5]:
input_cols = list(main_df.columns[0:-2])
target_col = 'attack_cat'

In [6]:
main_inputs , main_targets = main_df[input_cols].copy() , main_df[target_col].copy()

In [7]:
numeric_cols = main_inputs.select_dtypes(include = np.number).columns.tolist()[:]
categorical_cols = main_inputs.select_dtypes('object').columns.tolist()

In [8]:
categorical_cols.remove('sport')
categorical_cols.remove('dsport')

In [9]:
imputer = SimpleImputer(strategy = 'mean').fit(main_df[numeric_cols])
main_inputs[numeric_cols] = imputer.transform(main_inputs[numeric_cols])

In [10]:
scaler = MinMaxScaler().fit(main_df[numeric_cols])
main_inputs[numeric_cols] = scaler.transform(main_inputs[numeric_cols])

In [11]:
encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore').fit(main_inputs[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))

In [12]:
encoded_main_inputs = pd.DataFrame(encoder.transform(main_inputs[categorical_cols]).toarray(), columns=encoded_cols)

In [13]:
main_inputs = pd.concat([main_inputs,encoded_main_inputs],axis= 1)

In [14]:
del encoded_main_inputs ,main_df

In [15]:
main_inputs = main_inputs.drop(categorical_cols,axis=1)
# main_inputs = main_inputs.drop(['srcip', 'dstip', 'proto', 'state', 'service'],axis=1)

In [16]:
pickle.dump(main_targets,open('main_targets.pkl','wb'))

In [17]:
main_inputs

,sport,dsport,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,...,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,service_unkown
0,33661,1024,4.112267e-06,0.000037,0.000021,0.121569,0.114173,0.000000,0.000000,1.464196e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1464,53,1.273525e-07,0.000010,0.000012,0.121569,0.114173,0.000000,0.000000,8.715673e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3593,53,1.375953e-07,0.000009,0.000011,0.121569,0.114173,0.000000,0.000000,7.293329e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,49664,53,1.330429e-07,0.000010,0.000012,0.121569,0.114173,0.000000,0.000000,8.342890e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,32119,111,8.915697e-06,0.000040,0.000021,0.121569,0.114173,0.000000,0.000000,7.265069e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540038,33094,43433,9.936224e-06,0.000022,0.000125,0.121569,0.114173,0.000188,0.000363,4.085783e-06,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2540039,20848,21,4.154695e-05,0.000032,0.000024,0.121569,0.114173,0.000376,0.000363,1.460223e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2540040,21511,21,7.209987e-04,0.000126,0.000142,0.121569,0.114173,0.001316,0.001634,3.682097e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2540041,35433,80,2.504865e-04,0.000244,0.011329,0.121569,0.114173,0.000376,0.010350,2.086926e-06,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
main_targets

0            Benign
1            Benign
2            Benign
3            Benign
4            Benign
             ...   
2540038      Benign
2540039      Benign
2540040      Benign
2540041      Benign
2540042    Exploits
Name: attack_cat, Length: 2540043, dtype: object

In [19]:
main_inputs

,sport,dsport,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,...,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,service_unkown
0,33661,1024,4.112267e-06,0.000037,0.000021,0.121569,0.114173,0.000000,0.000000,1.464196e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1464,53,1.273525e-07,0.000010,0.000012,0.121569,0.114173,0.000000,0.000000,8.715673e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3593,53,1.375953e-07,0.000009,0.000011,0.121569,0.114173,0.000000,0.000000,7.293329e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,49664,53,1.330429e-07,0.000010,0.000012,0.121569,0.114173,0.000000,0.000000,8.342890e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,32119,111,8.915697e-06,0.000040,0.000021,0.121569,0.114173,0.000000,0.000000,7.265069e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540038,33094,43433,9.936224e-06,0.000022,0.000125,0.121569,0.114173,0.000188,0.000363,4.085783e-06,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2540039,20848,21,4.154695e-05,0.000032,0.000024,0.121569,0.114173,0.000376,0.000363,1.460223e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2540040,21511,21,7.209987e-04,0.000126,0.000142,0.121569,0.114173,0.001316,0.001634,3.682097e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2540041,35433,80,2.504865e-04,0.000244,0.011329,0.121569,0.114173,0.000376,0.010350,2.086926e-06,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
main_inputs.to_csv('Pp_main_df.csv')

In [ ]:
import pickle

In [ ]:
pickle.dump(imputer,open('imputer.pkl','wb'))
pickle.dump(scaler,open('scaler.pkl','wb'))
pickle.dump(encoder,open('encoder.pkl','wb'))
pickle.dump(main_targets,open('main_targets.pkl','wb'))